In [1]:
############ LOAD PACKAGES
import numpy as np
import pandas as pd
import napari
import tifffile as tf
import matplotlib.pyplot as plt
import time
from napari_animation import Animation


# import custum colormaps
from colormaps import *

# import custum functions
from STP_plotting import *
from STP_processing import *

# auto refreshes imported functions if source code changes:
%load_ext autoreload
%autoreload 2

In [2]:
# set-up vars
home_dir = "/Volumes/Data/"
# home_dir = "/mnt/labNAS/"

# in_path = home_dir+"Emily/STP_for_MAPseq/4_python_output/input_tifs/"
out_path = home_dir+"Emily/STP_for_MAPseq/4_python_output/output_figs/"

metadata = pd.read_csv("stp_metadata.csv")

In [3]:
# import reference brain

# # 220429
# ref_path = home_dir+"Emily/STP_for_MAPseq/2_fiji_adjusted/OMC_STeg_220429_b2_hand_straightened_asr.tif"
# ref_brain = "OMC_STeg_220429_b2"

# 220411
ref_path = home_dir+"Emily/STP_for_MAPseq/2_fiji_adjusted/OMC_STeg_220411_b3_hand_straightened_asr.tif"
ref_in = home_dir+"Emily/STP_for_MAPseq/3_brainreg_output/OMC_STeg_220411_straight_aligned_asr_aligned_10um/"
ref_brain = "OMC_STeg_220411_b1"

# # # 220208
# # ref_path = home_dir+"Emily/STP_for_MAPseq/2_fiji_adjusted/OMC_STeg_220208_b1_hand_straightened_asl.tif"
# ref_brain = "OMC_STeg_220208_b0"
ref_in = home_dir+"Emily/STP_for_MAPseq/3_brainreg_output/OMC_STeg_220208_straight_aligned_asr_aligned_10um/"

In [4]:
# based on ref brain, get path to brains
omc_meta = metadata[metadata["inj_site"]=="OMC"].reset_index(drop=True)

# get in_path based on reference brain
ref_name = ref_brain[4:-3]
in_path = home_dir+"Emily/STP_for_MAPseq/3_brainreg_output/"+ref_name+"_aligned/"

tif_paths = []
for brain in omc_meta["brain"]:
    if brain==ref_brain:
        tif_paths.append(ref_path)
    else:
        path = in_path+brain+"_brainreg_"+ref_name+"/downsampled_standard.tiff"
        tif_paths.append(path)

omc_meta["paths"] = tif_paths




In [5]:
%%time
# import images
start = time
tifs = []
for path in omc_meta["paths"]:
    tif = tf.imread(path)
    tifs.append(tif)
len(tifs)

CPU times: user 66.2 ms, sys: 1.31 s, total: 1.37 s
Wall time: 35.3 s


6

In [7]:
# create images for mmus and steg
mmus_avg = np.mean(np.array(tifs[:3]), axis=0)
steg_avg = np.mean(np.array(tifs[3:]), axis=0)


In [7]:
# # import example singing mouse
# # import hand-straight brain
# steg_to_st_tif = in_path+"norm_subtracted/STeg_220429_aligned/OMC_STeg_220208_b0_aligned_to_STeg_220429_NO_subtracted.tif"

# # import 230429 STeg brain- seems to be brighter than others???
# steg_hs_asr = home_dir+"Emily/STP_for_MAPseq/2_fiji_adjusted/OMC_STeg_220429_b2_hand_straightened_asr.tif"

# # # import example lab mouse
# # mmus_tif = home_dir+"Emily/STP_for_MAPseq/2_fiji_adjusted/OMC_MMus_220303_b1_hand_straightened_asr.tif"


# # import example lab mouse aligned to sm
# mmus_to_st_tif = in_path+"norm_subtracted/STeg_220429_aligned/OMC_MMus_220119_b0_aligned_to_STeg_220429_NO_subtracted.tif"


# # load tifs
# steg_to_st = tf.imread(steg_to_st_tif)
# # mmus = tf.imread(mmus_tif)
# steg = tf.imread(steg_hs_asr)
# mmus_to_st_b0 = tf.imread(mmus_to_st_tif)


In [ ]:
# import boundaries
# import aligned to hand-straight
boundaries_path = ref_in+"boundaries_downsize.tif"
boundaries = tf.imread(boundaries_path)




In [6]:
omc_meta["name"] = [brain[4:-3] for brain in omc_meta["brain"]]
omc_meta["contrast_factor"]= [7, 5, 3, 2, 3.5, 7]
omc_meta

,brain,species,inj_site,shape,paths,name,contrast_factor
0,OMC_MMus_220119_b0,MMus,OMC,NaN,/Volumes/Data/Emily/STP_for_MAPseq/3_brainreg_...,MMus_220119,7.0
1,OMC_MMus_220303_b1,MMus,OMC,NaN,/Volumes/Data/Emily/STP_for_MAPseq/3_brainreg_...,MMus_220303,5.0
2,OMC_MMus_220324_b2,MMus,OMC,NaN,/Volumes/Data/Emily/STP_for_MAPseq/3_brainreg_...,MMus_220324,3.0
3,OMC_STeg_220208_b0,STeg,OMC,NaN,/Volumes/Data/Emily/STP_for_MAPseq/3_brainreg_...,STeg_220208,2.0
4,OMC_STeg_220411_b1,STeg,OMC,NaN,/Volumes/Data/Emily/STP_for_MAPseq/2_fiji_adju...,STeg_220411,3.5
5,OMC_STeg_220429_b2,STeg,OMC,NaN,/Volumes/Data/Emily/STP_for_MAPseq/3_brainreg_...,STeg_220429,7.0


In [12]:
viewer = napari.Viewer()

# # add steg avg
# viewer.add_image(
#     steg_avg,
#     scale=[50,20,20]
# )

# # add mmus avg
# viewer.add_image(
#     mmus_avg,
#     scale=[50,20,20]
# )

# add individual tifs

# just load STeg_220411 and MMus_220303
# select_is = [1,4]

for i in range(len(tifs)):
    viewer.add_image(
        tifs[i],
        name=omc_meta.loc[i,"brain"],
        scale=[50,20,20]
    )
    # set contrast
    # adjust contrast limits, set max to max/contrast_factor
    cf = omc_meta.loc[i,"contrast_factor"]
    max_contrast = viewer.layers[omc_meta.loc[i,"brain"]].contrast_limits[1]
    viewer.layers[omc_meta.loc[i,"brain"]].contrast_limits = [0, max_contrast/cf]

    # custom contrast for 220411 and 220303
    # if i == 1:
    #     cf = 1301
    # elif i ==4:
    #     cf = 1423
    viewer.layers[omc_meta.loc[i,"brain"]].contrast_limits = [0, cf]


# add scale bar
viewer.scale_bar.visible = True
viewer.scale_bar.unit = "um"

In [15]:
# # add example image
# viewer.add_image(
#     boundaries,
#     scale=[50,20,20],
#     blending="additive"
# )

## save screen shot

In [33]:
# manipulate in napari until get orientation you want...
# take screenshot, specify size for highres
file_name = "mm2200324_alignto_st220411_z206_BS.png"
size = tifs[0].shape[1:]
viewer.screenshot(out_path+file_name, canvas_only=True, size=size)
print("done")

done


In [11]:
# given slice number save pic from each animal
z = 113
label="_AudR"

for i in range(omc_meta.shape[0]):
    brain_name = omc_meta.loc[i,"name"]
    
    # open viewer
    viewer = napari.Viewer()

    # load image
    viewer.add_image(
        tifs[i],
        name=brain_name,
        scale=[50,20,20],
    )
    
    # adjust contrast limits, set max to max/contrast_factor
    cf = omc_meta.loc[i,"contrast_factor"]

    max_contrast = viewer.layers[brain_name].contrast_limits[1]
    viewer.layers[brain_name].contrast_limits = [0, max_contrast/cf]

    # add scale bar
    viewer.scale_bar.visible = True
    viewer.scale_bar.unit = "um"

    # take screenshot
    file_name = brain_name+"_align_st220411_z"+str(z)+label+".png"
    size = tifs[i].shape[1:]
    viewer.screenshot(out_path+file_name, canvas_only=True, size=size)

    viewer.close()

## create animations

In [9]:
# try saving animation

animation = Animation(viewer)
viewer.update_console({"animation": animation})

# scroll through, pause on z45, 57, 100, 128, 167

viewer.dims.current_step = (0, 0, 0)
animation.capture_keyframe()

# # OMCi/OMCc - SM
# viewer.dims.current_step = (45, 0, 0)
# animation.capture_keyframe(steps=45)
# # pause here for 0.5 secs
# animation.capture_keyframe()
# animation.capture_keyframe(steps=5)

# # OMCi/OMCc - LM
# viewer.dims.current_step = (57, 0, 0)
# animation.capture_keyframe(steps=12)
# # pause here for 0.5 secs
# animation.capture_keyframe()
# animation.capture_keyframe(steps=5)

# decided to change to one OMCi/c site
viewer.dims.current_step = (50, 0, 0)
animation.capture_keyframe(steps=50)
# pause here for 0.5 secs
animation.capture_keyframe()
animation.capture_keyframe(steps=5)

# STR
viewer.dims.current_step = (100, 0, 0)
animation.capture_keyframe(steps=43)
# pause here for 0.5 secs
animation.capture_keyframe()
animation.capture_keyframe(steps=5)

# AudR, TH, SNr
viewer.dims.current_step = (128, 0, 0)
animation.capture_keyframe(steps=28)
# pause here for 1.5 secs
animation.capture_keyframe()
animation.capture_keyframe(steps=15)


# PAG, SCm, PG
viewer.dims.current_step = (167, 0, 0)
animation.capture_keyframe(steps=39)
# pause here for 1.5 secs
animation.capture_keyframe()
animation.capture_keyframe(steps=15)

# go through to end
viewer.dims.current_step = (280, 0, 0)
animation.capture_keyframe(steps=113)

animation.animate(out_path+"steg_220411_pauses.mp4", quality=9, fps=10, canvas_only=True)

Rendering frames...


100%|██████████| 374/374 [00:19<00:00, 18.74it/s]


In [45]:
# create loop to create animation for each individual animal

for i in range(omc_meta.shape[0]):
    brain_name = omc_meta.loc[i,"name"]
    
    # open viewer
    viewer = napari.Viewer()

    # load image
    viewer.add_image(
        tifs[i],
        name=brain_name,
        scale=[50,20,20],
    )
    
    # adjust contrast limits, set max to max/contrast_factor
    cf = omc_meta.loc[i,"contrast_factor"]

    max_contrast = viewer.layers[brain_name].contrast_limits[1]
    viewer.layers[brain_name].contrast_limits = [0, max_contrast/cf]

    # add scale bar
    viewer.scale_bar.visible = True
    viewer.scale_bar.unit = "um"

    # create animation
    animation = Animation(viewer)
    viewer.update_console({"animation": animation})

    viewer.dims.current_step = (0, 0, 0)
    animation.capture_keyframe()
    viewer.dims.current_step = (70, 0, 0)
    animation.capture_keyframe(steps=70)
    viewer.dims.current_step = (140, 0, 0)
    animation.capture_keyframe(steps=70)
    viewer.dims.current_step = (280, 0, 0)
    animation.capture_keyframe(steps=70)
    animation.animate(out_path+brain_name+"_aligned_steg_220411.mp4", quality=9, fps=10, canvas_only=True)

    viewer.close()

Rendering frames...


100%|██████████| 211/211 [00:13<00:00, 15.69it/s]


Rendering frames...


100%|██████████| 211/211 [00:11<00:00, 19.11it/s]


Rendering frames...


...

100%|██████████| 211/211 [00:11<00:00, 17.91it/s]


Rendering frames...


100%|██████████| 211/211 [00:11<00:00, 17.73it/s]


Rendering frames...


100%|██████████| 211/211 [00:10<00:00, 19.82it/s]
